# Homework week  3

In [1]:
import re

## Task 1

In [2]:
with open("./reputation.txt", "r") as f:
    lines = f.readlines()

In [3]:
len(lines)

77

In [4]:
compiled = re.compile(r"--\s(.+?)\srep\s(.+?)\s+=\s(\d+)")

In [5]:
compiled.search('-- 2012-08-30 rep +5    = 6         \n').groups()

('2012-08-30', '+5', '6')

In [6]:
data = []
for l in lines:
    res = compiled.search(l)
    if res:
        data.append(res.groups())

In [8]:
data

[('2012-08-30', '+5', '6'),
 ('2012-12-11', '+10', '16'),
 ('2013-03-20', '+10', '26'),
 ('2013-12-05', '0', '26'),
 ('2014-01-25', '0', '26'),
 ('2014-03-19', '+2', '28'),
 ('2014-05-11', '+2', '30'),
 ('2014-05-12', '+12', '42'),
 ('2014-06-12', '+10', '52'),
 ('2014-06-26', '+10', '62'),
 ('2014-07-05', '0', '62'),
 ('2014-09-02', '0', '62'),
 ('2014-09-03', '+10', '72'),
 ('2014-10-28', '0', '72'),
 ('2014-11-14', '+10', '82'),
 ('2014-11-18', '+2', '84'),
 ('2014-12-08', '+2', '86'),
 ('2014-12-09', '+10', '96'),
 ('2014-12-12', '+2', '98'),
 ('2014-12-24', '+10', '108'),
 ('2015-02-03', '0', '108'),
 ('2015-02-20', '+10', '118'),
 ('2015-03-28', '+10', '128'),
 ('2015-04-26', '+10', '138'),
 ('2015-05-05', '+10', '148'),
 ('2015-05-26', '+10', '158'),
 ('2015-05-27', '+20', '178'),
 ('2015-06-09', '0', '178'),
 ('2015-07-03', '+10', '188'),
 ('2015-07-06', '0', '188'),
 ('2015-07-22', '+110', '298'),
 ('2015-08-21', '+10', '308'),
 ('2015-09-07', '+10', '318'),
 ('2015-10-14', '-

## Task 1 bonus

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(data, columns=["date", "reputation_change", "reputation"])

In [11]:
df.head()

,date,reputation_change,reputation
0,2012-08-30,+5,6
1,2012-12-11,+10,16
2,2013-03-20,+10,26
3,2013-12-05,0,26
4,2014-01-25,0,26


In [12]:
df["reputation_change"] = df["reputation_change"].astype(int)

In [13]:
df["reputation"] = df["reputation"].astype(int)

In [14]:
df.tail()

,date,reputation_change,reputation
29,2015-07-06,0,188
30,2015-07-22,110,298
31,2015-08-21,10,308
32,2015-09-07,10,318
33,2015-10-14,-1,317


## Task 2

In [15]:
url = "https://raw.githubusercontent.com/planetsig/ufo-reports/master/csv-data/ufo-scrubbed-geocoded-time-standardized.csv"

In [16]:
data = pd.read_csv(url, header=None, nrows=100)

In [17]:
values = list(data[6])

In [18]:
value_re = re.compile(r"([\d\.\/]+).*?([a-zA-Z]+)")

In [19]:
less_re = re.compile(r"\<\s?(\d+)")

In [20]:
div_re = re.compile(r"(\d+)\/(\d+)")

In [21]:
div_re_sub = re.compile(r"\d+\/\d+")

In [22]:
def match(txt):
    txt = txt.replace("one", "1")
    # change less than to half number
    res = less_re.search(txt)
    if res:
        nr = float(res.group(1)) / 2
        txt = less_re.sub(str(nr), txt)
    
    # vulgar fractions to decimal fractions
    res = div_re.search(txt)
    if res:
        txt = div_re_sub.sub(str(int(res.group(1))  / int(res.group(2))), txt)
        
    # remove some words
    txt = re.sub("\s(to|or more)\s", " ", txt)
    txt = txt.replace("several", "5").replace("couple", "2")
    
    # main re
    res = value_re.search(txt)
    if res:
        groups = res.groups()
        
        # normalize units
        unit = re.sub("(minutes|mins|minute)", "min", groups[1])
        unit = re.sub("(seconds|secs)", "sec", unit)
        unit = re.sub("(hrs|hours|hour)", "hr", unit)
        groups = (groups[0], re.sub("(minutes|mins)", "min", unit))
        return groups
    else:
        return txt

In [23]:
result = []
for val in values:
    res = match(val)
    result.append((val, res[0], res[1]))

In [24]:
result

[('45 minutes', '45', 'min'),
 ('1-2 hrs', '1', 'hr'),
 ('20 seconds', '20', 'sec'),
 ('1/2 hour', '0.5', 'hr'),
 ('15 minutes', '15', 'min'),
 ('5 minutes', '5', 'min'),
 ('about 3 mins', '3', 'min'),
 ('20 minutes', '20', 'min'),
 ('3  minutes', '3', 'min'),
 ('several minutes', '5', 'min'),
 ('5 min.', '5', 'min'),
 ('3 minutes', '3', 'min'),
 ('30 min.', '30', 'min'),
 ('3 minutes', '3', 'min'),
 ('30 seconds', '30', 'sec'),
 ('20minutes', '20', 'min'),
 ('2 minutes', '2', 'min'),
 ('20-30 min', '20', 'min'),
 ('20 sec.', '20', 'sec'),
 ('45 minutes', '45', 'min'),
 ('20 minutes', '20', 'min'),
 ('one hour?', '1', 'hr'),
 ('5-6 minutes', '5', 'min'),
 ('1 minute', '1', 'min'),
 ('3 seconds', '3', 'sec'),
 ('30 seconds', '30', 'sec'),
 ('approx: 30 seconds', '30', 'sec'),
 ('5min', '5', 'min'),
 ('15 minutes', '15', 'min'),
 ('4.5 or more min.', '4.5', 'min'),
 ('3 minutes', '3', 'min'),
 ('30mins.', '30', 'min'),
 ('3 min', '3', 'min'),
 ('5 minutes', '5', 'min'),
 ('3 to 5 min', '

## Task 2 bonus

In [25]:
data2 = pd.read_csv(url, header=None, nrows=100, skiprows=100)

In [26]:
values = list(data2[6])

In [27]:
result = []
for val in values:
    res = match(val)
    result.append((val, res[0], res[1]))

In [28]:
result

[('10 sec', '10', 'sec'),
 ('1min', '1', 'min'),
 ('4 min.s', '4', 'min'),
 ('3 minutes', '3', 'min'),
 ('+5 minutes', '5', 'min'),
 ('2/min.', '2/', 'min'),
 ('3-5 minutes', '3', 'min'),
 ('15 min. apprx', '15', 'min'),
 ('less then a minute', 'l', 'e'),
 ('30 seconds', '30', 'sec'),
 ('00:43', '0', '0'),
 ('about5seconds', '5', 'sec'),
 ('10 mins', '10', 'min'),
 ('90 seconds', '90', 'sec'),
 ('2 minutes approx', '2', 'min'),
 ('30 minutes', '30', 'min'),
 ('2min', '2', 'min'),
 ('2 minute', '2', 'min'),
 ('5 min', '5', 'min'),
 ('30 minutes', '30', 'min'),
 ('10 minutes', '10', 'min'),
 ('12-15 seconds', '12', 'sec'),
 ('over 5 hours', '5', 'hr'),
 ('1 1/2 hr.', '1', 'hr'),
 ('3 minutes', '3', 'min'),
 ('45minutes', '45', 'min'),
 ('30min', '30', 'min'),
 ('2 - 3 minutes', '2', 'min'),
 ('45 min', '45', 'min'),
 ('20minutes', '20', 'min'),
 ('5 seconds', '5', 'sec'),
 ('15 seconds', '15', 'sec'),
 ('5:00', '5', ':'),
 ('10 sec', '10', 'sec'),
 ('3 minutes', '3', 'min'),
 ('1 hour', 

**Few errors**

In [29]:
import numpy as np

In [30]:
def compute_range(val):
    res = re.search(r"(\d+)-(\d+)", val)
    if res:
        return re.sub(r"\d+-\d+", str(np.mean([float(res.group(1)), float(res.group(2))])), val)
    return val

In [31]:
values = list(data[6])

result = []
for val in values:
    res = match(compute_range(val))
    result.append((val, res[0], res[1]))

In [32]:
result

[('45 minutes', '45', 'min'),
 ('1-2 hrs', '1.5', 'hr'),
 ('20 seconds', '20', 'sec'),
 ('1/2 hour', '0.5', 'hr'),
 ('15 minutes', '15', 'min'),
 ('5 minutes', '5', 'min'),
 ('about 3 mins', '3', 'min'),
 ('20 minutes', '20', 'min'),
 ('3  minutes', '3', 'min'),
 ('several minutes', '5', 'min'),
 ('5 min.', '5', 'min'),
 ('3 minutes', '3', 'min'),
 ('30 min.', '30', 'min'),
 ('3 minutes', '3', 'min'),
 ('30 seconds', '30', 'sec'),
 ('20minutes', '20', 'min'),
 ('2 minutes', '2', 'min'),
 ('20-30 min', '25.0', 'min'),
 ('20 sec.', '20', 'sec'),
 ('45 minutes', '45', 'min'),
 ('20 minutes', '20', 'min'),
 ('one hour?', '1', 'hr'),
 ('5-6 minutes', '5.5', 'min'),
 ('1 minute', '1', 'min'),
 ('3 seconds', '3', 'sec'),
 ('30 seconds', '30', 'sec'),
 ('approx: 30 seconds', '30', 'sec'),
 ('5min', '5', 'min'),
 ('15 minutes', '15', 'min'),
 ('4.5 or more min.', '4.5', 'min'),
 ('3 minutes', '3', 'min'),
 ('30mins.', '30', 'min'),
 ('3 min', '3', 'min'),
 ('5 minutes', '5', 'min'),
 ('3 to 5 m